In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [3]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v2 import *

In [4]:
from signal_model_utilities import force_vs_time as force22

In [5]:
%matplotlib widget

In [6]:
%reload_ext autoreload
%autoreload 2

### Take a look at the phase

In [7]:
def getPhases(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    x = bb.z2
    fft_ = np.fft.rfft(x,norm='ortho')
    freq_ = np.fft.rfftfreq(len(x), d=1./5000)
    return [np.angle(fft_[freq_==harm*3+1.5]) for harm in range(1,14)]
#     return np.sum(np.abs(fft_[freq_<200]))/1e9 

In [357]:
test_norms = np.array(Parallel(n_jobs=20)(delayed(getPhases)(i) for i in tqdm(range(10000))))







  0%|          | 0/10000 [00:00<?, ?it/s]





  1%|          | 80/10000 [00:00<00:15, 640.88it/s]





  1%|          | 120/10000 [00:00<00:28, 348.15it/s]





  2%|▏         | 200/10000 [00:00<00:31, 312.26it/s]





  3%|▎         | 280/10000 [00:01<00:37, 257.76it/s]





  4%|▎         | 360/10000 [00:01<00:42, 229.48it/s]





  4%|▍         | 440/10000 [00:01<00:44, 215.91it/s]





  5%|▌         | 520/10000 [00:02<00:48, 195.59it/s]





  6%|▌         | 600/10000 [00:02<00:49, 188.24it/s]





  7%|▋         | 680/10000 [00:03<00:56, 165.49it/s]





  8%|▊         | 760/10000 [00:03<00:52, 174.86it/s]





  8%|▊         | 840/10000 [00:04<00:54, 168.79it/s]





  9%|▉         | 920/10000 [00:04<00:55, 162.77it/s]





 10%|█         | 1000/10000 [00:05<00:45, 195.73it/s]





 11%|█         | 1080/10000 [00:05<00:37, 238.63it/s]





 12%|█▏        | 1160/10000 [00:05<00:32, 272.13it/s]





 12%|█▏        | 1240/10000 [00:05<00:28, 307.57it/s]





 13%|█▎        |

In [360]:
_,ax = plt.subplots(figsize=(9.2,3.2))
overall_norm = np.mean(test_norms)
ax.scatter(range(10000), test_norms/overall_norm)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [156]:
test_phase = np.array(Parallel(n_jobs=20)(delayed(getPhases)(i) for i in tqdm(range(10000)))).T[0]


 58%|█████▊    | 5840/10000 [01:31<01:00, 68.70it/s]


100%|██████████| 10000/10000 [02:51<00:00, 40.94it/s]

In [8]:
test_phasez2 = np.array(Parallel(n_jobs=20)(delayed(getPhases)(i) for i in tqdm(range(10000)))).T[0]

100%|██████████| 10000/10000 [00:30<00:00, 329.13it/s]


In [10]:
## force template
dist = 11.4
z_sep = -15.85
z_sep1 = -10
y_offset = 4.9
test = force_vs_time(dist,z_sep,200,3,"z",20, yuk_or_grav="yuk",offset_y=y_offset, 
                     alpha=1e8)
test1 = force_vs_time(dist,z_sep1,200,3,"z",20, yuk_or_grav="yuk",offset_y=y_offset, 
                     alpha=1e8)
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(5000, 5000)

_,ax = plt.subplots()
x = test[1]
x1 = test1[1]
fft = np.abs(np.fft.rfft(x))*scale
fft1 = np.abs(np.fft.rfft(x1))*scale
freq = np.fft.rfftfreq(len(x), d=1./5000)
angles = np.angle(np.fft.rfft(x))
phases_th = [angles[freq==freq_] for freq_ in 3*np.arange(14)]
# fft2 = np.abs(np.fft.rfft(x2))*scale
ax.semilogy(freq, fft, label='7.6 $\mu m$ sphere - Z=-15.7')
# ax.semilogy(freq, fft1, label='7.6 $\mu m$ sphere - Z=-10, X', alpha=0.5)
# ax.semilogy(freq, fft2, label='7.6 $\mu m$ sphere - Z force', alpha=0.5)
ax.set(xlim=(1,100), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()
print('Z force [N] at 36Hz.:', fft[freq==36])
print('sensitivity [alpha]:', 1e-17/fft[freq==36]*1e8)

Loading Gravity Data... Done!
51 1.99526231497e-05
Loaded Yukawa Force
Loading Gravity Data... Done!
51 1.99526231497e-05
Loaded Yukawa Force


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z force [N] at 36Hz.: [  6.00629579e-18]
sensitivity [alpha]: [  1.66491967e+08]


In [12]:
fig, ax = plt.subplots(3,4,figsize=(9,8.5), sharex=True)
i = 0
# _ = [a.hist(test_phase[j][:3000], label=str(np.arange(1,13)[j]*3)+'Hz.', density=True, bins=60, range=(-np.pi,np.pi)) for j,a in enumerate(ax.flatten())]
_ = [a.hist(test_phasez2[j][:3000], label=str(np.arange(1,13)[j]*3)+'Hz.', density=True, bins=60) for j,a in enumerate(ax.flatten())]
_ = [a.legend() for a in ax.flatten()]
_ = [a.axvline(phases_th[j], color='r', linestyle='dashed', linewidth=3) for j,a in enumerate(ax.flatten())]
# _ = [a.set(xlabel='phase [rad]') for a in ax[2][:].flatten()]
# _ = [a.set(ylabel='[a.u.]') for a in ax.flatten()[::4]]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Gravity results

In [6]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [7]:
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [8]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [9]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.2406092149549
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.6832937186789
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308067062
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.818416019317
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243984675
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.2380260183528
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478570203
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591716337
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737410096
********************

In [27]:
## likelihood calculator stuff
gravity_freqs = np.array([36, 33, 24, 18, 9])
gravity_freqs = np.array([15])
# gravity_freqs = np.array([21, 24, 27, 33])
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])

In [195]:
# noise estimation
data_bdfs2 = load_dir(dirname='/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/',
                         file_prefix='NoShaking_1', start_file=0, max_file=100)

## noise estimation
# data_bdfs2 = load_dir(dirname='/data/new_trap/20200320/Bead1/Shaking/Shaking378/',
#                          file_prefix='Shaking3', start_file=1000, max_file=100)

6905  files in folder
100  files loaded


In [28]:
noises_std = np.ones(len(gravity_freqs))

# gravity_freqs2 = gravity_freqs
# noises_std1 =[]
# for freq_ in gravity_freqs2:
#     tmp_list = []
#     for bb in data_bdfs2:
#         tmp_list.append(np.std(bb.response_at_freq2('z', drive_freq=freq_, bandwidth=bandwidth)))
#     noises_std1.append(np.std(tmp_list))
# noises_std = (np.array(noises_std1)/np.sqrt(100))**2
    
# gravity_freqs2 = gravity_freqs-1.5
# noises_std2 =[]
# for freq_ in gravity_freqs2:
#     tmp_list = []
#     for bb in data_bdfs2:
#         tmp_list.append(np.std(bb.response_at_freq2('z', drive_freq=freq_, bandwidth=bandwidth)))
#     noises_std2.append(np.std(tmp_list))

# noises_std = np.array([np.mean([n1, n2]) for n1,n2 in zip(noises_std1,noises_std2)])**2

In [29]:
noises_std, gravity_freqs

(array([ 1.]), array([15]))

In [30]:
i = 100
fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)

dist = 392.4-np.mean(bb.cant_pos[0])*50-7.6/2+1
z_sep = np.mean(height_cal)
y_offset = 4.9
stroke = np.std(bb.cant_pos[1])*np.sqrt(2)*50
G_lambda = 20

test_shaking378 = force_vs_time(dist,z_sep,stroke,bb.cant_freq,"z",G_lambda, 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
# test_shaking378 = force22(dist*1e-6,z_sep*1e-6,100e-6,3,"z",100e-6, offset_y=4e-6, yuk_or_grav="yuk",alpha=1e8)
shaking378_template = np.array(test_shaking378[1])

Loading Gravity Data... Done!
51 1.99526231497e-05
Loaded Yukawa Force


In [31]:
ll = likelihood_analyser.LikelihoodAnalyser()
def func2(i):
    print(i,'  ******************')
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, 2 * np.pi],
              'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
#     dist = 392.4-np.mean(bb.cant_pos[0])*50-7.6/2+1
#     z_sep = height_cal[i]
#     y_offset = 4.9
#     stroke = np.std(bb.cant_pos[1])*np.sqrt(2)*50
#     G_lambda = 20
#     test_shaking378 = force_vs_time(dist,z_sep,stroke,bb.cant_freq,"z",G_lambda, 
#                             yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
#     shaking378_template = np.array(test_shaking378[1])

    
#     jj = i//100
#     noises_std = np.array([sideband_36_std[jj], sideband_33_std[jj],sideband_12_std[jj],sideband_18_std[jj],sideband_21_std[jj]])**2
#     scales2 = np.array([sideband_36_mean2[jj]/np.mean(sideband_36_2), sideband_33_mean2[jj]/np.mean(sideband_33_2),
#                         sideband_24_mean2[jj]/np.mean(sideband_24_2),sideband_18_mean2[jj]/np.mean(sideband_18_2),
#                         sideband_9_mean2[jj]/np.mean(sideband_9_2)])

    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
                                          decimate=10, **fit_kwargs)
    
#     mm_tmp = ll.find_mle_multiHarmoincs_sideband(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
#                                           signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
#                                           decimate=10, **fit_kwargs)
    return mm_tmp.values[0]

In [32]:
mm_mle_All_lambda_20_noshaking_15Hz = np.array(Parallel(n_jobs=40)(delayed(func2)(i) for i in tqdm(range(1000))))

100%|██████████| 1000/1000 [00:20<00:00, 80.28it/s]


In [33]:
data = mm_mle_All_lambda_20_noshaking_15Hz
    
_,ax = plt.subplots()
avg = np.mean(data)
var = np.var(data)

range_hist = 20
# From that, we know the shape of the fitted Gaussian.
pdf_x = np.linspace(-range_hist,range_hist,100)
pdf_y = 1.0/np.sqrt(2*np.pi*var)*np.exp(-0.5*(pdf_x-avg)**2/var)

n, bins, patches = ax.hist(data, range=(-range_hist,range_hist), bins=30, density=True)
ax.set(xlim=(-range_hist,range_hist), xlabel=r'$\hat{\alpha} \,[10^{8}]$')
ax.plot(pdf_x,pdf_y,'k--')

print('mean: ', format(avg*1e8,"5.2e"))
print('std: ', format(np.std(data)*1e8,"5.2e"))
print('ste: ', format(np.std(data)/np.sqrt(len(data))*1e8,"5.2e"))

textstr = 'mean: '+format(avg*1e8,"5.2e")+'\nstd: '+format(np.std(data)*1e8,"5.2e")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  -1.93e+07
std:  6.05e+08
ste:  1.91e+07


Text(0.05, 0.95, 'mean: -1.93e+07\nstd: 6.05e+08')

In [19]:
format(np.std(data)*1e8/np.sqrt(1000),"5.2e")

'1.13e+07'

In [45]:
downsample = 100
data = mm_mle_All_lambda_20

alphas_mean = [np.mean(data[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))]
alphas_std = [np.std(data[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))]

alphas_mean2 = np.array([np.mean(mm_mle_All_lambda_20_sideband[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))])
alphas_std2 = np.array([np.std(mm_mle_All_lambda_20_sideband[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))])

_,ax = plt.subplots(figsize=(9.2,4))
rr_tmp = 15
ax.errorbar(time[::downsample]/60, np.array(alphas_mean), yerr=alphas_std, fmt='o', zorder=0, label='Noraml procedure')
ax.errorbar(time[::downsample]/60, np.array(alphas_mean2), yerr=alphas_std2, fmt='o', zorder=0, label='Shifted template')
ax.plot(np.arange(-100,10000,100)*10/60, np.arange(-100,10000,100)*0, 'r--', zorder=10)
ax.set(xlabel='Time [minute]', ylabel=r'$\alpha \,[10^{8}]$', ylim=(-rr_tmp,rr_tmp))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
def func_sideband(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    freq_tmp = 21
    return np.std(bb.response_at_freq2('z', drive_freq=freq_tmp+1.5, bandwidth=bandwidth))

In [35]:
sideband_21 = np.array(Parallel(n_jobs=40)(delayed(func_sideband)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [00:15<00:00, 665.78it/s]


In [24]:

sideband_36_mean = [np.mean(sideband_36[i*100:(i+1)*100]) for i in range(100)]
sideband_36_std = [np.std(sideband_36[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_33_mean = [np.mean(sideband_33[i*100:(i+1)*100]) for i in range(100)]
sideband_33_std = [np.std(sideband_33[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_24_mean = [np.mean(sideband_24[i*100:(i+1)*100]) for i in range(100)]
sideband_24_std = [np.std(sideband_24[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_18_mean = [np.mean(sideband_18[i*100:(i+1)*100]) for i in range(100)]
sideband_18_std = [np.std(sideband_18[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_9_mean = [np.mean(sideband_9[i*100:(i+1)*100]) for i in range(100)]
sideband_9_std = [np.std(sideband_9[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]


In [367]:
sideband_2_means = [np.mean(sideband_36_mean2),np.mean(sideband_33_mean2),np.mean(sideband_24_mean2),
                    np.mean(sideband_18_mean2),np.mean(sideband_9_mean2)]

In [305]:

sideband_36_mean2 = [np.mean(sideband_36_2[i*100:(i+1)*100]) for i in range(100)]
sideband_36_std2 = [np.std(sideband_36_2[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_36_mean3 = [np.mean(sideband_36_3[i*100:(i+1)*100]) for i in range(100)]
sideband_36_std3 = [np.std(sideband_36_3[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_33_mean2 = [np.mean(sideband_33_2[i*100:(i+1)*100]) for i in range(100)]
sideband_33_std2 = [np.std(sideband_33_2[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_24_mean2 = [np.mean(sideband_24_2[i*100:(i+1)*100]) for i in range(100)]
sideband_24_std2 = [np.std(sideband_24_2[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_18_mean2 = [np.mean(sideband_18_2[i*100:(i+1)*100]) for i in range(100)]
sideband_18_std2 = [np.std(sideband_18_2[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_9_mean2 = [np.mean(sideband_9_2[i*100:(i+1)*100]) for i in range(100)]
sideband_9_std2 = [np.std(sideband_9_2[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]


In [36]:
sideband_21_mean = [np.mean(sideband_21[i*100:(i+1)*100]) for i in range(100)]
sideband_21_std = [np.std(sideband_21[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_12_mean = [np.mean(sideband_12[i*100:(i+1)*100]) for i in range(100)]
sideband_12_std = [np.std(sideband_12[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_27_mean = [np.mean(sideband_27[i*100:(i+1)*100]) for i in range(100)]
sideband_27_std = [np.std(sideband_27[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_15_mean = [np.mean(sideband_15[i*100:(i+1)*100]) for i in range(100)]
sideband_15_std = [np.std(sideband_15[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

In [199]:
np.mean(mm_mle_All_lambda_20[:500]), np.mean(mm_mle_All_lambda_20[1000:])

(1.5214494881445397, 0.15990258686507483)

In [269]:
mm_mle_All_lambda_20_sideband = mm_mle_All_lambda_20_sideband*-1

In [472]:
from matplotlib import gridspec

downsample = 100
data = mm_mle_All_lambda_20_Harmonics8

alphas_mean = [np.mean(data[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))]
alphas_std = [np.std(data[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))]

alphas_mean2 = np.array([np.mean(mm_mle_All_lambda_20[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))])
alphas_std2 = np.array([np.std(mm_mle_All_lambda_20[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))])

fig = plt.figure(figsize=(9.5, 3.7)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[2.5, 1]) 
gs.update(wspace=0.0, hspace=0.05) # set the spacing between axes. 

rr_tmp = 15
ax0 = plt.subplot(gs[0])
ax0.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_mean), yerr=alphas_std, fmt='o', zorder=0)
ax0.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_mean2), yerr=alphas_std2, fmt='o', zorder=0)
ax0.plot(np.arange(-100,10000,100)*10/60, np.arange(-100,10000,100)*0, 'r--', zorder=10)
ax0.set(xlabel='Time [minute]', ylabel=r'$\alpha \,[10^{8}]$', ylim=(-rr_tmp,rr_tmp))

ax1 = plt.subplot(gs[1])
ax1.hist(alphas_mean, range=(-rr_tmp,rr_tmp), bins=19, orientation="horizontal", alpha=1.)
ax1.set_yticklabels([])
# ax1.set_xticklabels([])

ax1.set(ylim=(-rr_tmp,rr_tmp))
ax1.set_xlim(left=0, right=20)

# Create a duplicate of the original xaxis, giving you an additional axis object
ax2 = ax0.twinx()
# Set the limits of the new axis from the original axis limits
ax2.set_ylim(ax0.get_ylim())
ax2.set_yticklabels([])

# color='0.5'
# alpha=0.4
# sizes = ax1.figure.get_size_inches()
# size = 5.0 * np.sqrt(sizes[0]**2.0 + sizes[1]**2.0)
# ax1.figure.text(0.4, 0.3, 'Preliminary',
#                fontsize=size, color=color,
#                ha='center', va='center', alpha=alpha, rotation=0, zorder=0)

# ax1.plot(pdf_x_sig,pdf_y_sig,'k--')

# fig.savefig("wilson_alpha_lambda_20_downsample_100_3.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(1, -15.0, ''),
 Text(1, -10.0, ''),
 Text(1, -5.0, ''),
 Text(1, 0.0, ''),
 Text(1, 5.0, ''),
 Text(1, 10.0, ''),
 Text(1, 15.0, '')]

In [102]:
_,ax = plt.subplots(1,2,figsize=(9.2,4), sharey=True)
ax[0].scatter(mm_mle_All_lambda_20_sideband, mm_mle_All_lambda_20)
ax[1].scatter(mm_mle_All_lambda_20_old_sideband, mm_mle_All_lambda_20_old)
[a.set(xlabel=r'shifted template $\alpha \,[10^{8}]$', ylabel=r'$\alpha \,[10^{8}]$') for a in ax]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Text(0.5, 0, 'shifted template $\\alpha \\,[10^{8}]$'),
  Text(0, 0.5, '$\\alpha \\,[10^{8}]$')],
 [Text(0.5, 0, 'shifted template $\\alpha \\,[10^{8}]$'),
  Text(0, 0.5, '$\\alpha \\,[10^{8}]$')]]

In [37]:
from matplotlib import gridspec

# sideband_9_mean = [np.mean(sideband_9[i*100:(i+1)*100]) for i in range(100)]
# sideband_9_std = [np.std(sideband_9[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

fig = plt.figure(figsize=(9, 4)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[2.5, 1]) 

ax0 = plt.subplot(gs[0])
ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_36_mean, yerr=sideband_36_std, fmt='o')
# ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_36_mean2, yerr=sideband_36_std2, fmt='o')
ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_33_mean, yerr=sideband_33_std, fmt='o')
# ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_33_mean2, yerr=sideband_33_std2, fmt='o')
ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_24_mean, yerr=sideband_24_std, fmt='o')
# ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_24_mean2, yerr=sideband_24_std2, fmt='o')
ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_18_mean, yerr=sideband_18_std, fmt='o')
ax0.errorbar((np.arange(0,10000,100)+50)*10/60, sideband_9_mean, yerr=sideband_9_std, fmt='o')
# ax0.plot(np.arange(-100,10000,100)*10/60, np.arange(-100,10000,100)*0, 'r--')
# ax0.set(xlabel='Time [minute]', ylabel=r'$\alpha \,[10^{8}]$', ylim=(-15,15))

ax1 = plt.subplot(gs[1])
ax1.hist(sideband_36_mean, range=(17,28), bins=10, density=True)
# ax1.set(xlim=(-15,15), xlabel=r'$\hat{\alpha} \,[10^{8}]$')
# ax1.plot(pdf_x_sig,pdf_y_sig,'k--')

# fig.savefig("wilson_results.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 0.        ,  0.        ,  0.00909091,  0.24545455,  0.38181818,
         0.19090909,  0.05454545,  0.02727273,  0.        ,  0.        ]),
 array([ 17. ,  18.1,  19.2,  20.3,  21.4,  22.5,  23.6,  24.7,  25.8,
         26.9,  28. ]),
 <BarContainer object of 10 artists>)

### Systematics

In [75]:
n = 3
np.random.choice(3, n, replace=False)

array([1, 0, 2])

In [514]:
data = mm_mle_All_lambda_20_newNoise_oldHarmonics

systematics_means = [np.mean(data[:i]) for i in range(0,10000,100)]
systematics_stds = [np.std(data[:i]) for i in range(0,10000,100)]

systematics_means2 = []
for n in range(0,10000,100):
    index = np.random.choice(10000, n, replace=False)
    systematics_means2.append(np.mean(data[index]))
    
_,ax = plt.subplots(figsize=(9.2,4))
# ax.plot(np.arange(0,10000,100), systematics_stds,'o--')
ax.errorbar(time[::100]/60, systematics_means, yerr=systematics_stds/np.sqrt(np.arange(0,10000,100)), fmt='o', label='time ordered')
ax.errorbar(time[::100]/60, systematics_means2, yerr=systematics_stds/np.sqrt(np.arange(0,10000,100)), fmt='o', label='randomly sampled')
ax.plot(np.arange(0,10000,100)*10/60, np.arange(0,10000,100)*0, 'r--', zorder=10)
ax.set(xlabel='time [minutes]', ylabel=r'measured $\alpha [10^8]$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Figure of background phases

In [106]:
import numpy as np
import matplotlib.pyplot as plt

def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter

class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex='\pi'):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(multiple_formatter(self.denominator, self.number, self.latex))

In [387]:
def is_preliminary(ax, color='0.5', alpha=0.4):
    """ Add the 'Preliminary' legend into a plot
    
    ax: Must be a matplotlib.axes._subplots.AxesSubplot
        In practice ax is a Seaborn's plot. 
    """
    sizes = ax.figure.get_size_inches()
    size = 6.0 * np.sqrt(sizes[0]**2.0 + sizes[1]**2.0)
    ax.figure.text(0.5, 0.5, 'Preliminary',
                   fontsize=size, color=color,
                   ha='center', va='center', alpha=alpha, rotation=0, zorder=0)

In [400]:
# fig2, ax = plt.subplots(1,3,figsize=(13,3), sharey=True)

fig = plt.figure(figsize=(9, 3)) 
gs = gridspec.GridSpec(1, 3) 
gs.update(wspace=0.05, hspace=0.05) # set the spacing between axes. 

ax = [plt.subplot(gs[i]) for i in range(3)]


jj = [5,7,10,11]
_ = [a.hist(test_phase[j]-0.16, density=False, bins=60, range=(-np.pi,np.pi)) for a,j in zip(ax,jj)]

_ = [a.axvline(phases_th[j], color='r', linestyle='dashed', linewidth=3) for j,a in zip(jj,ax)]
# _ = [a.legend() for a in ax.flatten()]
_ = [a.set(xlabel='phase [rad]', title=str(np.arange(1,13)[j]*3)+'Hz.') for a,j in zip(ax,jj)]
ax[0].set(ylabel='[a.u.]')
for a in ax:
    a.grid(True)
    a.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
    a.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
    a.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
    a.set_yticks([])
plt.show()


is_preliminary(ax[0])
fig.savefig("wilson_phases.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Limit

In [544]:
def get_lim(L):
    print('  ******************', L)
    test_shaking3782 = force_vs_time(dist,z_sep,stroke,bb.cant_freq,"z",L, 
                                    yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
    shaking378_template2 = np.array(test_shaking3782[1])
    
    ll = likelihood_analyser.LikelihoodAnalyser()
    
    def func2_local(i):
        print(i,'  ******************')
        fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
        bb = BeadDataFile.BeadDataFile(fname=fname)

        fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
                  'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
                  'limit_phi': [0, 2 * np.pi],
                  'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
                  'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}

        jj = i//100
        noises_std = np.array([sideband_36_std[jj], sideband_33_std[jj],sideband_24_std[jj],sideband_18_std[jj],sideband_9_std[jj]])**2

        mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template2, scales=gravity_scales, 
                                              signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
                                              decimate=10, **fit_kwargs)

        return mm_tmp.values[0]
    
    mm_mle_All_lambda_local = np.array(Parallel(n_jobs=20)(delayed(func2_local)(i) for i in tqdm(range(10000))))
    return np.std(mm_mle_All_lambda_local)

In [ ]:
L = np.logspace(0.3, 2, 30)
limit_std = [get_lim(L=L_) for L_ in L]

  ****************** 1.99526231497
Loading Gravity Data... 










  0%|          | 0/10000 [00:00<?, ?it/s]

Done!
81 1.99526231497e-06
Loaded Yukawa Force











  0%|          | 20/10000 [00:24<3:27:33,  1.25s/it]








  0%|          | 40/10000 [00:35<2:51:40,  1.03s/it]








  1%|          | 60/10000 [00:39<2:08:17,  1.29it/s]








  1%|          | 80/10000 [00:41<1:35:34,  1.73it/s]








  1%|          | 100/10000 [00:43<1:12:29,  2.28it/s]








  1%|          | 120/10000 [00:45<54:53,  3.00it/s]  








  1%|▏         | 140/10000 [00:47<44:10,  3.72it/s]








  1%|▏         | 141/10000 [00:48<39:40,  4.14it/s]








  2%|▏         | 160/10000 [00:50<33:25,  4.91it/s]








  2%|▏         | 161/10000 [00:50<31:51,  5.15it/s]








  2%|▏         | 180/10000 [00:53<29:02,  5.63it/s]








  2%|▏         | 200/10000 [00:55<25:19,  6.45it/s]








  2%|▏         | 220/10000 [00:57<23:03,  7.07it/s]








  2%|▏         | 240/10000 [00:59<21:33,  7.55it/s]








  3%|▎         | 260/10000 [01:01<18:42,  8.68it/s]








  3%|▎         | 280/10000 [01:03<19:10,  8.45it/s]








  3%|▎         | 281/10

 33%|███▎      | 3260/10000 [02:50<02:21, 47.66it/s]








 33%|███▎      | 3280/10000 [02:51<02:23, 46.89it/s]








 33%|███▎      | 3300/10000 [02:51<02:21, 47.40it/s]








 33%|███▎      | 3320/10000 [02:52<02:21, 47.22it/s]








 33%|███▎      | 3340/10000 [02:52<02:14, 49.38it/s]








 34%|███▎      | 3360/10000 [02:53<03:15, 33.99it/s]








 34%|███▍      | 3380/10000 [02:54<03:12, 34.46it/s]








 34%|███▍      | 3400/10000 [02:54<02:54, 37.77it/s]








 34%|███▍      | 3420/10000 [02:55<02:50, 38.66it/s]








 34%|███▍      | 3425/10000 [02:55<02:58, 36.76it/s]








 34%|███▍      | 3440/10000 [02:55<02:47, 39.13it/s]








 35%|███▍      | 3460/10000 [02:55<02:24, 45.11it/s]








 35%|███▍      | 3480/10000 [02:56<02:25, 44.69it/s]








 35%|███▌      | 3500/10000 [02:56<02:33, 42.23it/s]








 35%|███▌      | 3520/10000 [02:57<02:42, 39.83it/s]








 35%|███▌      | 3540/10000 [02:57<02:45, 39.09it/s]








 36%|███▌      | 3560/10

...










  0%|          | 40/10000 [00:01<05:01, 33.06it/s]








  0%|          | 41/10000 [00:01<08:35, 19.31it/s]








  0%|          | 42/10000 [00:01<12:05, 13.73it/s]








  0%|          | 43/10000 [00:01<16:04, 10.33it/s]








  1%|          | 60/10000 [00:03<15:23, 10.76it/s]








  1%|          | 61/10000 [00:03<20:02,  8.26it/s]








  1%|          | 62/10000 [00:03<21:39,  7.65it/s]








  1%|          | 63/10000 [00:03<25:09,  6.58it/s]








  1%|          | 64/10000 [00:03<26:35,  6.23it/s]








  1%|          | 65/10000 [00:03<27:27,  6.03it/s]








  1%|          | 66/10000 [00:04<27:03,  6.12it/s]








  1%|          | 80/10000 [00:05<22:08,  7.47it/s]








  1%|          | 100/10000 [00:06<20:10,  8.18it/s]








  1%|          | 120/10000 [00:08<17:43,  9.29it/s]








  1%|▏         | 140/10000 [00:09<16:04, 10.22it/s]








  1%|▏         | 142/10000 [00:10<14:55, 11.01it/s]








  2%|▏         | 160/10000 [00:11<13:29, 12

 30%|██▉       | 2980/10000 [02:20<11:25, 10.24it/s]








 30%|███       | 3000/10000 [02:22<10:45, 10.84it/s]








 30%|███       | 3020/10000 [02:24<11:02, 10.53it/s]








 30%|███       | 3022/10000 [02:24<12:01,  9.67it/s]








 30%|███       | 3024/10000 [02:24<10:34, 10.99it/s]








 30%|███       | 3026/10000 [02:24<09:20, 12.45it/s]








 30%|███       | 3040/10000 [02:26<09:39, 12.01it/s]








 30%|███       | 3042/10000 [02:26<12:15,  9.46it/s]








 30%|███       | 3044/10000 [02:26<12:51,  9.02it/s]








 31%|███       | 3060/10000 [02:29<13:48,  8.37it/s]








 31%|███       | 3080/10000 [02:30<12:01,  9.60it/s]








 31%|███       | 3100/10000 [02:33<12:54,  8.91it/s]








 31%|███       | 3120/10000 [02:34<11:16, 10.17it/s]








 31%|███       | 3122/10000 [02:34<09:50, 11.64it/s]








 31%|███▏      | 3140/10000 [02:37<12:03,  9.48it/s]








 31%|███▏      | 3142/10000 [02:37<10:15, 11.14it/s]








 32%|███▏      | 3160/10

 58%|█████▊    | 5800/10000 [05:29<04:48, 14.55it/s]








 58%|█████▊    | 5820/10000 [05:30<03:57, 17.59it/s]








 58%|█████▊    | 5840/10000 [05:31<03:28, 19.97it/s]








 59%|█████▊    | 5860/10000 [05:31<03:13, 21.42it/s]








 59%|█████▉    | 5880/10000 [05:32<03:08, 21.81it/s]








 59%|█████▉    | 5900/10000 [05:33<03:11, 21.44it/s]








 59%|█████▉    | 5920/10000 [05:34<03:12, 21.20it/s]








 59%|█████▉    | 5940/10000 [05:36<03:33, 18.99it/s]








 59%|█████▉    | 5943/10000 [05:36<03:23, 19.89it/s]








 60%|█████▉    | 5960/10000 [05:37<03:36, 18.66it/s]








 60%|█████▉    | 5980/10000 [05:38<03:33, 18.85it/s]








 60%|██████    | 6000/10000 [05:39<03:26, 19.34it/s]








 60%|██████    | 6002/10000 [05:39<03:36, 18.49it/s]








 60%|██████    | 6020/10000 [05:40<03:41, 17.96it/s]








 60%|██████    | 6022/10000 [05:40<04:41, 14.14it/s]








 60%|██████    | 6040/10000 [05:42<04:50, 13.62it/s]








 60%|██████    | 6042/10

 84%|████████▎ | 8360/10000 [09:01<00:59, 27.46it/s]








 84%|████████▍ | 8380/10000 [09:02<00:57, 28.41it/s]








 84%|████████▍ | 8383/10000 [09:02<00:57, 28.26it/s]








 84%|████████▍ | 8386/10000 [09:02<00:59, 27.05it/s]








 84%|████████▍ | 8389/10000 [09:02<01:00, 26.69it/s]








 84%|████████▍ | 8392/10000 [09:02<01:08, 23.56it/s]








 84%|████████▍ | 8395/10000 [09:02<01:06, 24.14it/s]








 84%|████████▍ | 8400/10000 [09:03<01:05, 24.44it/s]








 84%|████████▍ | 8420/10000 [09:03<00:58, 27.01it/s]








 84%|████████▍ | 8440/10000 [09:04<00:54, 28.63it/s]








 84%|████████▍ | 8443/10000 [09:04<00:55, 28.26it/s]








 85%|████████▍ | 8460/10000 [09:04<00:49, 31.02it/s]








 85%|████████▍ | 8480/10000 [09:05<00:42, 35.84it/s]








 85%|████████▍ | 8484/10000 [09:05<00:42, 36.08it/s]








 85%|████████▍ | 8488/10000 [09:05<00:41, 36.71it/s]








 85%|████████▌ | 8500/10000 [09:05<00:36, 40.59it/s]








 85%|████████▌ | 8520/10

  ****************** 2.99131445041
Loading Gravity Data... 

In [ ]:
_, ax = plt.subplots(figsize=(7,5))
ax.loglog(L[2:], np.array(limit_mean[2:])*1e8,'--o', label=r'Wilson - mean')
ax.loglog(L[2:], np.array(limit[2:])*1e8,'--o', label=r'Wilson - std')
ax.set(xlabel='$\lambda [\mu m]$', ylabel=r'$\alpha$')
ax.legend()

### Alpha vs. height for Alex

In [157]:
## extract initial values

i = 100
fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)

dist = 392.4-np.mean(bb.cant_pos[0])*50-7.6/2+1
z_sep = -15.85
y_offset = 4.9
stroke = np.std(bb.cant_pos[1])*np.sqrt(2)*50
G_lambda = 20

In [187]:
dist

14.607115214873897

In [ ]:
gravity_scales, gravity_freqs

In [346]:
alpha_vs_dist = []
for dist in np.arange(5, 25):
    
    z_sep = -15.85

    print(dist,'  ******************')
    
    test_shaking378 = force_vs_time(dist,z_sep,stroke,bb.cant_freq,"z",G_lambda, 
                                    yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
    shaking378_template = np.array(test_shaking378[1])
        
    mm_mle_All_lambda_local = np.array(Parallel(n_jobs=20)(delayed(func2)(i) for i in tqdm(range(1000))))
    alpha_vs_dist.append(mm_mle_All_lambda_local)
alpha_vs_dist = np.array(alpha_vs_dist)

5   ******************
Loading Gravity Data... 






  0%|          | 0/1000 [00:00<?, ?it/s]

Done!
30 0.0001
Loaded Yukawa Force







  0%|          | 1/1000 [00:00<16:25,  1.01it/s]




  2%|▏         | 20/1000 [00:01<11:29,  1.42it/s]




  4%|▍         | 40/1000 [00:02<08:05,  1.98it/s]




  6%|▌         | 60/1000 [00:03<05:39,  2.77it/s]




  8%|▊         | 80/1000 [00:03<03:56,  3.89it/s]




 10%|█         | 100/1000 [00:03<02:45,  5.44it/s]




 12%|█▏        | 120/1000 [00:04<01:57,  7.51it/s]




 14%|█▍        | 140/1000 [00:04<01:22, 10.41it/s]




 16%|█▌        | 160/1000 [00:04<00:59, 14.09it/s]




 18%|█▊        | 180/1000 [00:04<00:43, 18.74it/s]




 19%|█▉        | 188/1000 [00:04<00:33, 24.26it/s]




 20%|██        | 200/1000 [00:05<00:27, 29.13it/s]




 22%|██▏       | 220/1000 [00:05<00:21, 36.67it/s]




 24%|██▍       | 240/1000 [00:05<00:16, 45.88it/s]




 25%|██▍       | 249/1000 [00:05<00:14, 51.96it/s]




 26%|██▌       | 260/1000 [00:05<00:12, 58.79it/s]




 28%|██▊       | 280/1000 [00:05<00:11, 65.44it/s]




 30%|███       | 300/1000 [00:06<00:10, 69.71it/s]




 32%|███▏  

6   ******************
Loading Gravity Data... 

...

8   ******************
Loading Gravity Data... 






  0%|          | 0/1000 [00:00<?, ?it/s]






Done!
30 0.0001
Loaded Yukawa Force


  4%|▍         | 40/1000 [00:00<00:03, 244.24it/s]




  5%|▍         | 48/1000 [00:00<00:07, 127.61it/s]




  8%|▊         | 80/1000 [00:00<00:06, 152.40it/s]




 12%|█▏        | 120/1000 [00:00<00:06, 143.60it/s]




 16%|█▌        | 160/1000 [00:01<00:06, 133.89it/s]




 20%|██        | 200/1000 [00:01<00:06, 128.11it/s]




 24%|██▍       | 240/1000 [00:01<00:06, 125.83it/s]




 28%|██▊       | 280/1000 [00:02<00:05, 127.33it/s]




 32%|███▏      | 320/1000 [00:02<00:05, 123.33it/s]




 36%|███▌      | 360/1000 [00:02<00:05, 122.05it/s]




 40%|████      | 400/1000 [00:03<00:04, 123.37it/s]




 44%|████▍     | 440/1000 [00:03<00:04, 117.59it/s]

KeyboardInterrupt: 

In [ ]:
stds_alpha = np.std(alpha_vs_dist, axis=1)

_,ax = plt.subplots()
ax.plot(np.arange(5, 25), stds_alpha/np.sqrt(1000)*1e8, 'o--')
ax.set(xlabel=r'$\Delta$X [$\mu$m]',ylabel=r'Sensitivity $[\alpha]$', title=r'$\Delta$Z = -15.85$\mu$m')

In [186]:
means_alpha = np.mean(alpha_vs_sep2, axis=1)
stds_alpha = np.std(alpha_vs_sep2, axis=1)

_,ax = plt.subplots()
ax.plot(np.arange(-15, -6), stds_alpha/np.sqrt(1000)*1e8, 'o--')
ax.set(xlabel=r'$\Delta$Z [$\mu$m]',ylabel=r'Sensitivity $[\alpha]$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$\\Delta$Z [$\\mu$m]'), Text(0, 0.5, 'Sensitivity $[\\alpha]$')]

In [61]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

In [76]:
time -= time[0]

In [78]:
_,ax = plt.subplots(figsize=(9.5,4))
# ax.scatter(range(1000), height_cal[::10])
ax.scatter(np.arange(100)*10, time[:100])
np.mean(z_distance)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-15.85910039296383